### 1.导入LLM models

In [1]:
# config
ProjectPath = "/remote-home/yy/lzd/DocDoc" 
MODEL_PATH = "/remote-home/share/LLM_model/chatglm3-6b"
TOKENIZER_PATH = "/remote-home/share/LLM_model/chatglm3-6b"
EMBEDDING_PATH = "/remote-home/share/LLM_model//bge-large-zh"
OUTPUT_PATH = "./output"

In [2]:
# 1.导入LLM models
from LLMs import ChatGLM

LLM = ChatGLM()
LLM.load_model(MODEL_PATH = MODEL_PATH, TOKENIZER_PATH = TOKENIZER_PATH)

print(LLM("HELLO!")) # say hello

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Hello! How can I assist you today?


In [3]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import numpy as np
embeddings = HuggingFaceEmbeddings(model_name = EMBEDDING_PATH,
                                    model_kwargs = {'device': "cuda"})
query_result = embeddings.embed_query("DocDoc") 
print("embedding query.shape=", np.array(query_result).shape)
print("embedding import completed")

embedding query.shape= (1024,)
embedding import completed


### 2.上传文件 loader

In [4]:
# 2.上传文件 loader
# 提取word文档的文本
from docx import Document  # pip install python-docx 而不是 docx
filename = '汨罗江洪道治理报告书（2021）.docx'
filepath = ProjectPath + "/test/testfile/" + filename
doc = Document(filepath)   # load word
# print(doc.paragraphs) # 打印内存地址
texts = ""
for paragraph in doc.paragraphs:
    # print(paragraph.text)  # 文本
    texts = texts + paragraph.text + "\n" # 拼接文本
print(texts)  # texts -> str
print("step2 completed")

目  录
概 述	11
1.总论	66
1.1 编制依据	66
1.1.1 国家法律、法规、政策	66
1.3.2 地方法规、政策、规划	77
1.3.3 相关技术规范	88
1.3.4 其它资料	98
1.2 环境影响因素识别及评价因子筛选	99
1.2.1 环境影响因素识别	99
1.2.2 评价因子筛选	109
1.3 环境功能区划与评价标准	1010
1.3.1 环境功能区划	1010
1.3.2 评价标准	1111
1.3.3污染物排放标准	1312
1.4 评价工作等级及评价范围	1414
1.4.1 评价工作等级	1414
1.4.2 评价范围	1716
1.5 评价内容及评价重点	1716
1.5.1 评价内容	1816
1.5.2 评价重点	1817
1.6 环境保护目标	1817
1.6.1环境功能保护目标	1817
1.6.2环境敏感保护目标	1817
2.工程概况	2120
2.1 汨罗江现状调查	2120
2.1.1 汨罗江洪道现状及项目由来	2120
2.1.2治理目标	2120
2.1.3治理的必要性	2221
2.2 拟建项目基本情况	2322
2.3工程内容及规模	2422
2.4 项目实施方案	2423
2.4.1河道整治方案	2423
2.4.2工程总体布置	2523
2.4.3 洪道疏挖	2625
2.4.4 淤泥处置	2626
2.5 工程占地及土石方平衡	2827
2.5.1 工程占地及拆迁	2827
2.5.2土石方平衡	2827
2.6 施工组织	2828
2.6.1施工条件	2828
2.6.2主体工程施工	2928
2.6.3施工总体布置	3332
2.6.4施工总进度	3433
2.6.5工程机械设备计划表	3433
3.工程分析	3535
3.1施工工艺及工程影响因素分析	3535
3.1.1 施工工艺流程分析	3535
3.1.2 工程影响因素分析	3737
3.2施工期主要污染源	3737
3.2.1 废气	3737
3.2.2 废水	3838
3.2.3 噪声	3939
3.2.4 固体废物产生与处理处置	3939
3.2.5 生态环境影响因素	4040
3.3 营运期主要污染影响	4141
4.环境现状调查与评价	4343
4.1 自然环境概况	4343
4.1.1 地理位置	4343
4.1.2 地形

### 文档分割 spliter

In [5]:
# 加入到 flowcontrol
# 3.分割文档 spliter
from langchain.text_splitter import SpacyTextSplitter
from langchain.docstore.document import Document
text_splitter = SpacyTextSplitter(
        separator = "\n\n",
        pipeline = "zh_core_web_sm",  # 加载 spacy 的中文模型
        max_length = 1_000_000,
        chunk_size=1000,    # 最大切割字符：1000个
        chunk_overlap = 200)
 
texts_splited = text_splitter.split_text(texts)
# texts_splited -> list[str]

# 构造 langchain Document 对象
text_Document_list = [Document(page_content=text, metadata={"from": "filename or book.txt"}) for text in texts_splited]
# text_Document_list -> list[Document()]


In [14]:
print(texts_splited[5])

干流长253.

2km，流域面积5543km2。

汨罗江下游洪道从汨罗市滨江拦河闸下游1km～周家垅段总长约15.33km，该河段紧邻汨罗江入洞庭湖口，洞庭湖洪水顶托导致的水流速度减缓加之多年未对河道进行疏浚，导致汨罗江下游洪道河床目前淤积严重，此外洪道内卡口、矶头等碍洪建筑物阻水，致使洪水位逐年抬高，洪水持续时间延长，洪道行洪能力下降，河势不断演变，水情日趋恶化洪涝灾害常有发生。

1983年汩罗江上游降暴雨，三天降雨达200mm，由于洞庭湖湖水顶托，卡口阻水严重，上游水位急剧雍高，造成汩罗市罗江、湖溪、幸福等堤垸相继溃决，长乐镇以下江河横溢，一片汪洋，倒塌房屋1.78万间，受灾人口6万余人，受灾耕地17.18万亩，经济损失7900万元，更为严重的是跨越汩罗江京广铁路的南渡大桥洪水漫桥，行车中断10余小时，经大力抢救才保住了大桥的安全。

这次灾害给国家和下游人民造成了巨大的经济损失，社会影响很大。

1993年7月汩罗市普降暴雨，降雨量达421mm，这次山洪暴发虽未溃垸，但受渍淹农田近12万亩，受灾人口约1.97万人，直接经济损失2310万元。

1998年更是多处溃垸，尾闾地区一片汪洋。




2020年7月，连日暴雨后汨罗江水位暴涨，各控制站均超警戒水位0.6m以上，导致京广线铁路桥大部分桥墩被淹没，汨罗江风光带积水严重，汨罗江湿地公园及屈子文化核心景区等地均被大水冲刷，严重危及沿线城镇、重要交通干线及景区安全。

汨罗江洪道泥沙淤积十分严重，行洪能力下降，且堤防抗洪标准低，洪涝灾害常有发生，洪水对汨罗江洪道（滨江闸～周家垅段）

两岸乡镇的人民生命财产造成威胁。

洪道整治后，将保障汨罗江河道行洪安全和周边群众生命财产安全，对改善汨罗江水质，维护洞庭湖流域生态安全，改善人居生存环境，促进区域经济发展等方面产生积极影响。

日益恶化的洪涝灾害对汨罗江沿线经济、交通、旅游和农业发展等均带来很大的影响，且随着社会的不断发展、政策要求的不断提高以及人民群众对物质文化要求的不断提升，加剧了洪涝灾害带来的影响力，因此对汨罗江下洪道进行整治势在必行，且刻不容缓。


In [8]:
for _ in texts_splited:
    print(len(_))

996
979
987
698
989
897
970
911
955
962
962
938
856
955
954
825
842
957
947
878
963
973
839
979
928
994
875
982
860
967
968
973
916
974
971
979
948
943
930
997
972
990
989
962
908
938
972
998
976
976
905
942
983
949
948
998
953
959
853
971
851
977
941
982
933
985
973
979
970
969
917
767
909
964
986
918
987
951
919
966
974
970
974
946
987
979
980
981
982
937
941
905
935
947
996
1000
993
970
979
934
988
883
818
949
951
954
963
990
971
975
898
345


### 4.向量化 embedding

In [6]:
vs_path = "demo-vs"  # vector_db 存储地址
from langchain.vectorstores import FAISS # FASIS
# docs = embeddings.embed_documents(texts1) # -> list[向量]
vector_store = FAISS.from_documents(text_Document_list, embeddings) # 建立 向量数据库
vector_store.save_local(vs_path)  # 存储到vsdb中
print(vector_store)

### 5.向量检索 retriever

In [7]:
vector_store = FAISS.load_local(vs_path, embeddings)  # 加载 vector_db
related_docs_with_score = vector_store.similarity_search_with_score(query="在撰写《环评报告》的“结论”章节时，需要注意什么？", k=10) # 相似性搜索，搜索前K条知识并打分
print(related_docs_with_score)


[(Document(page_content='工程的不利环境影响主要表现在施工期，在落实本环评报告提出的环境保护减缓措施，所产生的不利影响可以得到有效控制，并降至环境能接受的程度；工程的环境效益和社会效益显著。\n\n从环保的角度分析，项目实施可行。\n\n\n\n1.总论\n1.1 编制依据\n1.1.\n\n1 国家法律、法规、政策\n(1)《中华人民共和国环境保护法》（2015年1月1日）。\n\n\n(2)《中华人民共和国环境影响评价法》（2018年12月29日修订施行）。\n\n\n(3)《中华人民共和国水土保持法》（2011年3月1日）。\n(\n\n4)《中华人民共和国水污染防治法》（2018年1月1日修订施行）。\n(\n\n5)《中华人民共和国大气污染防治法》（2018年12月29日修订施行）。\n\n\n(6)《中华人民共和国环境噪声污染防治法》（2018年12月29日修订施行）。\n(7)《中华人民共和国固体废物污染环境防治法》（2020年9月1日修订施行）。\n(8) 《中华人民共和国水法》（2016年7月修订）。\n(9)《中华人民共和国土壤污染防治法》（2019 年1 月1 日起施行）。\n\n\n(10)《建设项目环境影响评价分类管理名录》（2018年4月28日修订）。\n\n\n(11)《产业结构调整指导目录（2019年本）》。\n(12)《环境影响评价公众参与办法》（2019年1月1日施行）。\n\n\n(13) 《关于划定并严守生态保护红线的若干意见》（2017.\n\n2.7）。\n\n\n(14)\n\n《关于加强城市建设项目环境影响评价监督管理工作的通知》，（环发[2008]70号），2008年9月18日。\n\n\n(15)《水污染防治行动计划》（2015年4月16日）。\n(16)《土壤污染防治行动计划》（2016年5月31日）。\n\n\n(17)《大气污染防治行动计划》（2013年9月10日）。\n(18) 《关于进一步加强水生生物资源保护严格环境影响评价管理的通知》（环发[2013]86号）。\n（19）《国际湿地保护公约》，1982.\n\n3.12。\n（20）《中国湿地保护行动计划》，2000.11。\n\n\n（21）\n\n《湿地保护管理规定》国家林业局令第32号，2018.\n\n1.1修订实施。\

### 6.模板拼接 prompt = content + query

In [11]:
contents = ""
for pack in related_docs_with_score:
    doc, socre = pack
    content = doc.page_content
    print("检索到的知识=%s, from=%s, socre=%.3f"%(content, doc.metadata.get("from"), socre))
    print("*"*100)
    contents += content  # 知识拼接

检索到的知识=工程的不利环境影响主要表现在施工期，在落实本环评报告提出的环境保护减缓措施，所产生的不利影响可以得到有效控制，并降至环境能接受的程度；工程的环境效益和社会效益显著。

从环保的角度分析，项目实施可行。



1.总论
1.1 编制依据
1.1.

1 国家法律、法规、政策
(1)《中华人民共和国环境保护法》（2015年1月1日）。


(2)《中华人民共和国环境影响评价法》（2018年12月29日修订施行）。


(3)《中华人民共和国水土保持法》（2011年3月1日）。
(

4)《中华人民共和国水污染防治法》（2018年1月1日修订施行）。
(

5)《中华人民共和国大气污染防治法》（2018年12月29日修订施行）。


(6)《中华人民共和国环境噪声污染防治法》（2018年12月29日修订施行）。
(7)《中华人民共和国固体废物污染环境防治法》（2020年9月1日修订施行）。
(8) 《中华人民共和国水法》（2016年7月修订）。
(9)《中华人民共和国土壤污染防治法》（2019 年1 月1 日起施行）。


(10)《建设项目环境影响评价分类管理名录》（2018年4月28日修订）。


(11)《产业结构调整指导目录（2019年本）》。
(12)《环境影响评价公众参与办法》（2019年1月1日施行）。


(13) 《关于划定并严守生态保护红线的若干意见》（2017.

2.7）。


(14)

《关于加强城市建设项目环境影响评价监督管理工作的通知》，（环发[2008]70号），2008年9月18日。


(15)《水污染防治行动计划》（2015年4月16日）。
(16)《土壤污染防治行动计划》（2016年5月31日）。


(17)《大气污染防治行动计划》（2013年9月10日）。
(18) 《关于进一步加强水生生物资源保护严格环境影响评价管理的通知》（环发[2013]86号）。
（19）《国际湿地保护公约》，1982.

3.12。
（20）《中国湿地保护行动计划》，2000.11。


（21）

《湿地保护管理规定》国家林业局令第32号，2018.

1.1修订实施。




（22）《国家湿地公园管理办法》林湿发〔2017〕150号，2017.

12.27。
（23）, from=filename or book.txt, so

In [12]:
# 模板拼接
query="帮我撰写的“结论”章节。结构要求：1. 项目概况 2. 区域环境质量现状评价结论 3. 环境影响分析结论 4. 环境可行性分析结论  5. 公众参与结论  6. 综合结论"
template = f"已知：{contents}, 请你根据你的知识和我提供给你的信息，{query}"
print(template)

已知：工程的不利环境影响主要表现在施工期，在落实本环评报告提出的环境保护减缓措施，所产生的不利影响可以得到有效控制，并降至环境能接受的程度；工程的环境效益和社会效益显著。

从环保的角度分析，项目实施可行。



1.总论
1.1 编制依据
1.1.

1 国家法律、法规、政策
(1)《中华人民共和国环境保护法》（2015年1月1日）。


(2)《中华人民共和国环境影响评价法》（2018年12月29日修订施行）。


(3)《中华人民共和国水土保持法》（2011年3月1日）。
(

4)《中华人民共和国水污染防治法》（2018年1月1日修订施行）。
(

5)《中华人民共和国大气污染防治法》（2018年12月29日修订施行）。


(6)《中华人民共和国环境噪声污染防治法》（2018年12月29日修订施行）。
(7)《中华人民共和国固体废物污染环境防治法》（2020年9月1日修订施行）。
(8) 《中华人民共和国水法》（2016年7月修订）。
(9)《中华人民共和国土壤污染防治法》（2019 年1 月1 日起施行）。


(10)《建设项目环境影响评价分类管理名录》（2018年4月28日修订）。


(11)《产业结构调整指导目录（2019年本）》。
(12)《环境影响评价公众参与办法》（2019年1月1日施行）。


(13) 《关于划定并严守生态保护红线的若干意见》（2017.

2.7）。


(14)

《关于加强城市建设项目环境影响评价监督管理工作的通知》，（环发[2008]70号），2008年9月18日。


(15)《水污染防治行动计划》（2015年4月16日）。
(16)《土壤污染防治行动计划》（2016年5月31日）。


(17)《大气污染防治行动计划》（2013年9月10日）。
(18) 《关于进一步加强水生生物资源保护严格环境影响评价管理的通知》（环发[2013]86号）。
（19）《国际湿地保护公约》，1982.

3.12。
（20）《中国湿地保护行动计划》，2000.11。


（21）

《湿地保护管理规定》国家林业局令第32号，2018.

1.1修订实施。




（22）《国家湿地公园管理办法》林湿发〔2017〕150号，2017.

12.27。
（23）结论	130127
10.

1 结论	130127
10.1.1 项

### 7.生成回答 response

In [13]:
response = LLM(template)
response

'结论\n\n1. 项目概况\n本项目位于湖南省岳阳市汨罗市，主要对汨罗江进行疏浚和航道整治。项目实施过程中，将严格遵守国家法律法规和政策，确保项目的合规性和可持续发展。\n\n2. 区域环境质量现状评价结论\n根据项目实施前后的监测数据和分析，项目对周边环境的水质、土壤和生态系统的影响总体较小，符合环境质量标准要求。项目实施后，环境质量得到了一定程度的改善，有助于提升周边居民的生态环境质量。\n\n3. 环境影响分析结论\n本项目在施工期和运营期对周边环境可能产生一定影响，主要表现为水体污染、土壤污染和噪声污染。项目实施时已采取了一系列减缓措施，力求将环境影响降至可接受程度。项目运营期通过加强环境保护管理，可有效控制污染物排放，进一步降低环境影响。\n\n4. 环境可行性分析结论\n综合考虑项目区域环境质量现状、环境影响分析结果和环境可行性措施，本项目在实施过程中能够有效降低对周边环境的影响，且在项目运营期能够通过加强环境保护管理，进一步降低环境影响。因此，项目具有较好的环境可行性。\n\n5. 公众参与结论\n本项目在决策过程中充分考虑了公众的意见和需求，通过公众参与程度较高的环评程序，确保了项目实施符合社会公众利益。项目实施过程中，也将积极倾听公众意见，及时处理环境问题，确保项目的顺利实施。\n\n6. 综合结论\n本项目在遵循国家法律法规和政策的前提下，充分考虑了区域环境质量和公众利益，实施过程中将采取有效的环境保护措施，降低对周边环境的影响。项目具有较好的环境可行性，可以为岳阳市汨罗市的环境改善和经济发展做出贡献。在项目实施过程中，需加强环境保护管理，确保项目的可持续发展。'

### 生成 pdf/word/txt

In [14]:
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import Table, SimpleDocTemplate, Paragraph, Image  # 报告内容相关类
from reportlab.lib.pagesizes import letter, A4
from reportlab.pdfbase import pdfmetrics
from reportlab.lib import  colors
from reportlab.pdfbase.ttfonts import TTFont

pdfmetrics.registerFont(TTFont('宋体', './fonts/simsun.ttc')) # 字体注册

# 获取所有样式表
style = getSampleStyleSheet()
# 获取普通样式
ct = style['Normal']
ct.fontName = '宋体'

response_pdf = response.replace("\n", "<br/>") # 转换为 pdf 的换行符

In [17]:
# 生成pdf文件
pdfname = "output_spacy01.pdf"
OUTPUT_PATH = "./output"
outputpath = OUTPUT_PATH + "/"+ pdfname 

doc = SimpleDocTemplate(outputpath, pagesize=letter)
doc.build([Paragraph(response_pdf, ct)])
print(f"{pdfname}生成完毕！")
print(f"文件位于{outputpath}")

output_spacy01.pdf生成完毕！
文件位于./output/output_spacy01.pdf


In [18]:
response_pdf

'结论<br/><br/>1. 项目概况<br/>本项目位于湖南省岳阳市汨罗市，主要对汨罗江进行疏浚和航道整治。项目实施过程中，将严格遵守国家法律法规和政策，确保项目的合规性和可持续发展。<br/><br/>2. 区域环境质量现状评价结论<br/>根据项目实施前后的监测数据和分析，项目对周边环境的水质、土壤和生态系统的影响总体较小，符合环境质量标准要求。项目实施后，环境质量得到了一定程度的改善，有助于提升周边居民的生态环境质量。<br/><br/>3. 环境影响分析结论<br/>本项目在施工期和运营期对周边环境可能产生一定影响，主要表现为水体污染、土壤污染和噪声污染。项目实施时已采取了一系列减缓措施，力求将环境影响降至可接受程度。项目运营期通过加强环境保护管理，可有效控制污染物排放，进一步降低环境影响。<br/><br/>4. 环境可行性分析结论<br/>综合考虑项目区域环境质量现状、环境影响分析结果和环境可行性措施，本项目在实施过程中能够有效降低对周边环境的影响，且在项目运营期能够通过加强环境保护管理，进一步降低环境影响。因此，项目具有较好的环境可行性。<br/><br/>5. 公众参与结论<br/>本项目在决策过程中充分考虑了公众的意见和需求，通过公众参与程度较高的环评程序，确保了项目实施符合社会公众利益。项目实施过程中，也将积极倾听公众意见，及时处理环境问题，确保项目的顺利实施。<br/><br/>6. 综合结论<br/>本项目在遵循国家法律法规和政策的前提下，充分考虑了区域环境质量和公众利益，实施过程中将采取有效的环境保护措施，降低对周边环境的影响。项目具有较好的环境可行性，可以为岳阳市汨罗市的环境改善和经济发展做出贡献。在项目实施过程中，需加强环境保护管理，确保项目的可持续发展。'